http://mogile.web.fc2.com/spark/rdd-programming-guide.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import findspark 
findspark.init()
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession


In [2]:
conf = SparkConf().setAppName('new_app').setMaster('local')
sc = SparkContext(conf=conf)

並列化されたコレクション

In [3]:
data = [1,2,3,4,5]
distData = sc.parallelize(data)
distData

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:194

In [4]:
distData = sc.parallelize(data,5)
distData

ParallelCollectionRDD[1] at parallelize at PythonRDD.scala:194

In [5]:
path_and_file = './text/text.txt'
rdd_file = sc.textFile(path_and_file)

In [6]:
rdd_file.map(lambda s : len(s)).reduce(lambda a,b:a+b)

3876

textFileを含むSparkのファイルベースの全ての入力メソッドは、ディレクトリ、圧縮ファイルおよびワイルドカードでの実行もサポートします。例えば、textFile("/my/directory"), textFile("/my/directory/*.txt") および textFile("/my/directory/*.gz") を使うことができます。

In [7]:
rdd = sc.parallelize(range(1, 4)).map(lambda x: (x, "a" * x))
rdd.collect()

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

In [8]:
rdd.saveAsSequenceFile("a")
sc.sequenceFile("a")


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsSequenceFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/ko_kaku/Desktop/00_self_learning/spark/a already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withSco

rdd.saveAsは、いろんなものに保存できます

 クロージャーの理解 

ローカル vs. クラスタモード

In [9]:
counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)


Counter value:  0


In [10]:
lines = sc.textFile('./text/text.txt')
pairs = lines.map(lambda s: (s, '!!!!!!!!!!!!!!!!!!!'))
counts = pairs.reduceByKey(lambda a, b: a + b)
counts.collect()

[('Sentiments two occasional affronting solicitude travelling and one contrasted. Fortune day out married parties. Happiness remainder joy but earnestly for off. Took sold add play may none him few. If as increasing contrasted entreaties be. Now summer who day looked our behind moment coming. Pain son rose more park way that. An stairs as be lovers uneasy. ',
  '!!!!!!!!!!!!!!!!!!!'),
 ('',
  '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'),
 ('Improve ashamed married expense bed her comfort pursuit mrs. Four time took ye your as fail lady. Up greatest am exertion or marianne. Shy occasional terminated insensible and inhabiting gay. So know do fond to half on. Now who promise was justice new winding. In finished on he speaking suitable advanced if. Boy happiness sportsmen say prevailed offending concealed nor was provision. Provided so as doub

# Spark SQL

# ストリーミングプログラミング

In [23]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [24]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [25]:
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

NameError: name 'explode' is not defined